# Jérôme IV – YOLOv8 Fine-Tuning on Roboflow Datasets

**Author:** Éloïse Delerue

**Laboratory:** Université Paris Sciences & Lettres

**Contact:** eloise.delerue@psl.eu

**Date:** January 15th, 2026

## 0. Read Me

### 0.1. Overview

**Jérôme IV** is part of a research series focused on applying computer vision to art history. Its primary goal is to fine-tune a YOLOv8 object detection model from [Ultralytics](https://github.com/ultralytics/ultralytics) to specifically recognize human figures within paintings. It serves as the first step of a complete object detection workflow, preceding **Jérôme II** (inference and deployment).

The script writing relied essentially on the following tutorials:
- [YOLOv8 – Comment l’utiliser ? Meilleur Tutoriel](https://inside-machinelearning.com/yolov8/)
- [Fine-tuning YOLO for Custom Object Detection: A Complete Guide](https://medium.com/@amresh.kumar11/fine-tuning-yolo-for-custom-object-detection-a-complete-guide-6ce3724ce9f1)
- [Fine-tuning YOLOS on a custom dataset for object detection](https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/YOLOS/Fine_tuning_YOLOS_for_object_detection_on_custom_dataset_(balloon).ipynb)
- [Fine-tune YOLOv8 models for custom use cases with the help of FiftyOne](https://colab.research.google.com/github/voxel51/fiftyone/blob/v0.24.1/docs/source/tutorials/yolov8.ipynb#scrollTo=e9f710e1-3d00-45d4-96a6-ccc7dee4e138)

**Please note that it is designed to run in Google Colab.**

### 0.2. Data

The training utilizes a robust, consolidated dataset created by merging two distinct sources from **Roboflow**:

- **[people-in-paintings](https://universe.roboflow.com/roboflow-100/people-in-paintings):** A specialized collection of art-based human annotations.

- **[peopleArt](https://universe.roboflow.com/peopleartyolo/peopleart-4fnuw):** A supplementary dataset to enhance the model's ability to generalize across different artistic styles.

To ensure consistency, all labels were harmonized to a single class: **"person"**. The final dataset includes geometric and photometric data augmentation (blur, grayscale, mosaic) to prevent overfitting and improve detection in varied lighting and compositions.

**[IMPORTANT] Roboflow API Key Required:** To run the data acquisition cells, you must use your own Roboflow API key.

1.   Log in to **your Roboflow account**.
2.   Navigate to **Settings** > **Workspaces** > **API Keys**.
3.   Copy your **Private API Key** and paste it into the `api_key` field in the code below.

In [ ]:
rf = Roboflow(api_key="YOUR_API_KEY_HERE")

### 0.3. Model

The present training uses the **YOLOv8n** model as a base. You may adjust epochs (number of iterations), imgsz (image size, 640 px for the provided dataset), batch, lr0, or other hyperparameters in section 4.1.

### 0.4. Training & Evaluation

YOLOv8 automatically validates the model during the training, at the end of each epoch. Training results are logged in **runs/detect/train/**, and several metrics can be found in **runs/detect/val/**. Some have been displayed in section 5.

### 0.5. Model Export

At the end of training, export and download the fine-tuned model **jerome.pt** in the files.

## 1. Setup & Libraries Import

In [ ]:
# Installing and Importing Ultralytics Library
!pip install -q ultralytics roboflow
from roboflow import Roboflow
from ultralytics import YOLO
import ultralytics
ultralytics.checks()

Ultralytics 8.3.245 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/112.6 GB disk)


In [ ]:
# Importing Libraries for File and Directory Management
import os
import random
import shutil

# Importing Libraries for File and Data Management
from google.colab import files
import yaml
import numpy as np

# Importing Libraries for Image Processing
from PIL import Image
import requests
from io import BytesIO
import cv2
from google.colab.patches import cv2_imshow

# Importing Libraries for Visualisation
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

## 2. Helper Function

In [ ]:
def copy_dataset_files(source_path, dest_path, subset):
    """
    Consolidates image and label files from a source dataset into a destination directory.

    Args:
        source_path: Root directory of the source dataset.
        dest_path: Root directory where the merged dataset is stored.
        subset: Data split to process (e.g., 'train', 'valid', or 'test').
    """
    # Define source and destination paths for images and labels based on the subset
    images_src = os.path.join(source_path, subset, "images")
    labels_src = os.path.join(source_path, subset, "labels")
    images_dest = os.path.join(dest_path, subset, "images")
    labels_dest = os.path.join(dest_path, subset, "labels")

    # Copy Image Files
    for img_file in os.listdir(images_src):
        src_file = os.path.join(images_src, img_file)
        dest_file = os.path.join(images_dest, img_file)

        # Collision management: if a file with the same name exists, append a suffix
        counter = 1
        while os.path.exists(dest_file):
            name, ext = os.path.splitext(img_file)
            dest_file = os.path.join(images_dest, f"{name}_{counter}{ext}")
            counter += 1

        # Perform the actual file transfer to the merged dataset directory
        shutil.copy(src_file, dest_file)

    # Copy Label Files
    for label_file in os.listdir(labels_src):
        src_file = os.path.join(labels_src, label_file)
        dest_file = os.path.join(labels_dest, label_file)

        # Collision management: ensure labels are renamed identically to their images
        counter = 1
        while os.path.exists(dest_file):
            name, ext = os.path.splitext(label_file)
            dest_file = os.path.join(labels_dest, f"{name}_{counter}{ext}")
            counter += 1

        # Transfer the annotation files (YOLO format .txt)
        shutil.copy(src_file, dest_file)

## 3. Data Import & Merging

### 3.1. Dataset Downloads

In [ ]:
# people-in-paintings Dataset Acquisition from Roboflow
project = rf.workspace("roboflow-100").project("people-in-paintings")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
# peopleArt Dataset Acquisition from Roboflow
project = rf.workspace("peopleartyolo").project("peopleart-4fnuw")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
# Paths to source datasets
dataset1_path = "people-in-paintings-2"
dataset2_path = "peopleArt-1"
merged_dataset_path = "people-in-paintings-merged"

### 3.2. Data Harmonisation

In [ ]:
# Update the 'data.yaml' configuration for the people-in-paintings-2 dataset
data_yaml_path = os.path.join(dataset1_path, "data.yaml")
with open(data_yaml_path, "r") as f:
    data = yaml.safe_load(f)

# Unify class names by replacing 'Human' with 'person'
data["names"] = ["person"]

with open(data_yaml_path, "w") as f:
    yaml.dump(data, f, sort_keys=False)

print(f"Fichier {data_yaml_path} mis à jour : 'names' est maintenant ['person'].")

### 3.3. Dataset Merging & YAML Configuration

In [ ]:
# Create the directory structure for the merged dataset
os.makedirs(os.path.join(merged_dataset_path, "train", "images"), exist_ok=True)
os.makedirs(os.path.join(merged_dataset_path, "train", "labels"), exist_ok=True)
os.makedirs(os.path.join(merged_dataset_path, "valid", "images"), exist_ok=True)
os.makedirs(os.path.join(merged_dataset_path, "valid", "labels"), exist_ok=True)
os.makedirs(os.path.join(merged_dataset_path, "test", "images"), exist_ok=True)
os.makedirs(os.path.join(merged_dataset_path, "test", "labels"), exist_ok=True)

In [ ]:
# Execute the merging process for both source datasets across all subsets
for subset in ["train", "valid", "test"]:
    copy_dataset_files(dataset1_path, merged_dataset_path, subset)
    copy_dataset_files(dataset2_path, merged_dataset_path, subset)

In [ ]:
# Generate the 'data.yaml' configuration file for the merged dataset
merged_data_yaml = {
    "train": "./train/images",
    "val": "./valid/images",
    "test": "./test/images",
    "nc": 1,           # Number of classes
    "names": ["person"] # List of class names
}

# Write the configuration to a YAML file in the merged dataset directory
with open(os.path.join(merged_dataset_path, "data.yaml"), "w") as f:
    yaml.dump(merged_data_yaml, f, sort_keys=False)

## 4. Hyperparameters Configuration, Model Training & Validation

### 4.1. Hyperparameters Configuration

In [ ]:
# Hyperparameters
HYPERPARAMS = {
    "model_type": "yolov8n.pt",
    "epochs": 100,
    "imgsz": 640,
    "batch_size": 8,
    "learning_rate": 0.01,
    "final_lr": 0.01,
    "patience": 10,
    "augment": True,
}

### 4.2. Model Loading

In [ ]:
# Load the model
model = YOLO(HYPERPARAMS["model_type"])

### 4.3. Model Training & Validation

In [ ]:
# Train the model
results = model.train(
    data="/content/people-in-paintings-merged/data.yaml",
    epochs=HYPERPARAMS["epochs"],
    imgsz=HYPERPARAMS["imgsz"],
    batch=HYPERPARAMS["batch_size"],
    lr0=HYPERPARAMS["learning_rate"],
    patience=HYPERPARAMS["patience"],
    augment=HYPERPARAMS["augment"],
)

Ultralytics 8.3.245 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/people-in-paintings-merged/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plo

## 5. Model Evaluation

In [ ]:
# Filtered validation specifically for the "person" class (index 0)
metrics = model.val(
    data="people-in-paintings-merged/data.yaml",
    split="test",
    classes=[0]
)

# Display the evaluation results
print(f"mAP50-95 for the 'Person' class: {metrics.box.map}")

Ultralytics 8.3.245 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2354.6±1189.8 MB/s, size: 128.3 KB)
val: Scanning /content/people-in-paintings-merged/test/labels.cache... 244 images, 11 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 244/244 482.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 3.5it/s 4.6s
                   all        244        924      0.689      0.524      0.602      0.298
Speed: 2.4ms preprocess, 3.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/runs/detect/val2
mAP50-95 pour la classe Person: 0.29791960667756323


## 6. Model Export

In [ ]:
# Save the model
model.save("jerome.pt")